In [40]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import csv
import numpy as np

with open("indeed_scraped_data_science.csv", "r",encoding="ISO-8859-1") as f:
    reader = csv.reader(f, delimiter=',')
    text1 = [(row[8]) for row in reader if row[0]=='1']


with open("indeed_scraped_data_science.csv", "r",encoding="ISO-8859-1") as f:
    reader = csv.reader(f, delimiter=',')
    text2 = [(row[8]) for row in reader if row[0]=='2']
    
with open("indeed_scraped_data_science.csv", "r",encoding="ISO-8859-1") as f:
    reader = csv.reader(f, delimiter=',')
    text3 = [(row[8]) for row in reader if row[0]=='3']



In [41]:
import string
import nltk
import re
import csv
import ast

from re import compile
#for sorting dictionary
import operator
from nltk.corpus import stopwords
from nltk.collocations import *

def tokenize(text):
    tokens=[]
    # write your code here
    stop_words = stopwords.words('english')
    text = text.lower()
    #print(text)
    #pattern = r'\w'
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip(string.punctuation) for token in tokens]
    
    tokens = [token for token in tokens if re.match("^[A-Za-z_-]*$", token)]
    #print(tokens)
    # to remove punctuations from begging and starting of the tokens 
    
    # now removing extra empty characters from tokens
    tokens = [token.strip() for token in tokens if token.strip()!='']
    tokens = [token for token in tokens if len(token)>1]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

In [42]:
# get a Keras tokenizer

MAX_NB_WORDS=8000
# documents are quite long in the dataset
MAX_DOC_LEN=1000

tokenizer1 = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer1.fit_on_texts(text1)

tokenizer2 = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer2.fit_on_texts(text2)

tokenizer3 = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer3.fit_on_texts(text3)

input_tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
input_tokenizer.fit_on_texts(text1[0])

with open("project_lead.csv", "r",encoding="ISO-8859-1") as f:
    reader=csv.reader(f, delimiter=',')
    rows=[(row[8]) for row in reader]
    
#print(rows[0])
#input_tokens = []

input_tokens = tokenize(rows[10])

   
#print(input_tokens)

In [43]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df=pd.DataFrame.from_dict(tokenizer1.word_counts, orient="index")
df.columns=['freq']
print(df.head())
df=df['freq'].value_counts().reset_index()
df.columns=['word_freq','count']

df=df.sort_values(by='word_freq')
#print(tokenizer1.word_counts)
print(df.head())

df=pd.DataFrame.from_dict(tokenizer2.word_counts, orient="index")
df.columns=['freq']
print(df.head())
df=df['freq'].value_counts().reset_index()
df.columns=['word_freq','count']

df=df.sort_values(by='word_freq')
#print(tokenizer1.word_counts)
print(df.head())


df=pd.DataFrame.from_dict(tokenizer3.word_counts, orient="index")
df.columns=['freq']
print(df.head())
df=df['freq'].value_counts().reset_index()
df.columns=['word_freq','count']

df=df.sort_values(by='word_freq')
#print(tokenizer1.word_counts)
print(df.head())

             freq
description   323
fis            23
provides      190
financial     425
software      637
   word_freq  count
1          1   3129
0          2   3476
2          3   1239
3          4   1201
5          5    529
          freq
'product    25
package    122
planner     23
united      32
parcel       4
   word_freq  count
0          1   7634
1          2   4459
2          3   1506
3          4   1253
4          5    642
              freq
'taught          6
students        64
the           2589
introduction     6
to            2587
   word_freq  count
0          1   5065
1          2   1752
2          3    783
3          4    521
4          5    380


In [44]:
import string
import nltk
import re
import csv
import ast
#for initial data filtering 
import preprocessing
from re import compile
#for sorting dictionary
import operator
from nltk.corpus import stopwords
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()




def dateToSum(filename):
    temp = 0
    with open(filename, "r",encoding="ISO-8859-1") as f:
        reader = csv.reader(f, delimiter=',')

        rows = [row[7] for row in reader]
        tot = len(rows)
        # print(tot)
        for rro in rows:
            if len(rro) > 0:
                exp_len = ast.literal_eval(rro)
                # print(exp_len)
                for t in exp_len:
                    tokens = nltk.word_tokenize(t)
                    for i in range(len(tokens)):
                        if tokens[i] == "NA":
                            tokens[i] = ""
                        elif tokens[i] == "Present":
                            tokens[i] = "2018"
                    rex = compile('[^0-9]')
                    filteredData = [x for x in tokens if not rex.match(x)]

                    if len(filteredData) == 2:
                        temp = (int(filteredData[1]) - int(filteredData[0])) + temp

        print("\n\nAverage Work Experience : ")
        print(round(temp/tot))


In [45]:
def performance_evaluate(input_file):
    trigrams_list =[]
    bigrams_list=[]
    # write your code here
    with open(input_file, "r",encoding="ISO-8859-1") as f:
        reader=csv.reader(f, delimiter=',')
        # row[1] or second column is header of resume which states the current job position.
        # Although we did data scraping for particular job positions, current job positions varied a lot
        # in initial analysis phase- so even after scraping resumes for one particular position, we are making sure the
        # actual job postion for which analysis will be done.
        rows=[(row[1], row[5], row[4], row[10], row[3],row[8]) for row in reader]
    
    
    
   
    #print(row_len)
    for i in rows:
        token_list = tokenize(i[0])
        tmp_list = list(nltk.trigrams(token_list))
        tmp_big_list = list(nltk.bigrams(token_list))
        #print(tmp_big_list)
        for j in tmp_list:
            trigrams_list.append(j)
        for l in tmp_big_list:
            bigrams_list.append(l)
    #print(bigrams_list)      
    
    #Here for job titles we cannot use NLTK's trigram collocation finder as it will filter out some important details.
    #so finding it manually.
    trigram_freq={}
    bigram_freq={}
    
    for k in trigrams_list:
        if k in trigram_freq:
            trigram_freq[k]+=1
        else:
            trigram_freq[k]=1
            
    for k in bigrams_list:
        if k in bigram_freq:
            bigram_freq[k]+=1
        else:
            bigram_freq[k]=1
        
    # uncomment below print line to see the frequency for each trigram
    # Sorting the dictionary below to easily see and get the top values when needed. For trigrams
    sorted_freq = sorted(trigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    #print(sorted_freq)
    #print("Sorted by frequency - trigrams from heading\n\nSorted by Frequency Bigrams from heading")
    #for bigrams
    bi_sorted_freq = sorted(bigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    #print(bi_sorted_freq)
    
    # now the top most value is what we are looking for. and all our analysis will be for that position.
    desired_position = sorted_freq[0][0]
    print("\n\nThis analysis is for ",sorted_freq[0][0])
    print("\n##############################################\n")
    #--------------------------------------------
    # now for 5th column i.e. job titles.
    #--------------------------------------------
    exp_trigrams_list =[]
    exp_bigrams_list=[]
    for i in rows:
        #second param is job exp
        exp_list = tokenize(i[1])
        #print(exp_list[0])
        tmp_list = list(nltk.trigrams(exp_list))
        tmp_big_list = list(nltk.bigrams(exp_list))
        
        for j in tmp_list:
            exp_trigrams_list.append(j)
        for l in tmp_big_list:
            exp_bigrams_list.append(l)
    
    exp_trigram_freq={}
    exp_bigram_freq={}
    
    for k in exp_trigrams_list:
        if k in exp_trigram_freq:
            exp_trigram_freq[k]+=1
        else:
            exp_trigram_freq[k]=1
       
    for k in exp_bigrams_list:
        if k in exp_bigram_freq:
            exp_bigram_freq[k]+=1
        else:
            exp_bigram_freq[k]=1
      
    
    exp_sorted_freq = sorted(exp_trigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    #print(exp_sorted_freq)
    print("'input resume' can have more chances to become  ",desired_position," if it has following job experience terms\n")
    
    for i in range(0,10):
        print(exp_sorted_freq[i][0])
    print("\n\nOR----####----####----####\n\n")
    #for bigrams
    exp_bi_sorted_freq = sorted(exp_bigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    #print(exp_bi_sorted_freq)
    for i in range(0,10):
        print(exp_bi_sorted_freq[i][0])
    
    
    print("\n##############################################\n")
    #--------------------------------------------
    # now for 4th column i.e. skills.
    #--------------------------------------------
    skills_list = []
    for i in rows:
        token_list = tokenize(i[2])
        
        for j in token_list:
            skills_list.append(j)
        
    #print(len(skills_list))
    skill_dict={}
    for k in skills_list:
        if k in skill_dict:
            skill_dict[k]+=1
        else:
            skill_dict[k]=1
            
        
    # uncomment below print line to see the frequency for each trigram
    # Sorting the dictionary below to easily see and get the top values when needed. For trigrams
    sorted_freq = sorted(skill_dict.items(), key=operator.itemgetter(1),reverse=True)
    print("\n\nThese are preferred skills for the position.\n")
    custom = ["years","year","na","less","than","and"]
    for i in range(0,35):
        if(sorted_freq[i][0] not in custom):
            print(sorted_freq[i][0], sorted_freq[i][1])
            
            
            
    print("###############  Description top words  ################3")
    
    skills_list = []
    for i in rows:
        token_list = tokenize(i[5])
        
        for j in token_list:
            skills_list.append(j)
    
    
        
    #print(len(skills_list))
    skill_dict={}
    for k in skills_list:
        if k in skill_dict:
            skill_dict[k]+=1
        else:
            skill_dict[k]=1
            
        
    # uncomment below print line to see the frequency for each trigram
    # Sorting the dictionary below to easily see and get the top values when needed. For trigrams
    sorted_freq = sorted(skill_dict.items(), key=operator.itemgetter(1),reverse=True)
    print("\n\nThese are preferred description words for the position.\n")
    custom = ["years","year","na","less","than","and","using"]
    return_list = []
    for i in range(0,len(sorted_freq)-5):
        return_list.append(sorted_freq[i])
        
    for i in range(0,35):
        if(sorted_freq[i][0] not in custom):
            print(sorted_freq[i][0], sorted_freq[i][1])
    
    
    print("\n##############################################\n")
    #--------------------------------------------
    # now for 10th column i.e. skills.
    #--------------------------------------------
    skills_list = []
    trigrams_list = []
    for i in rows:
        token_list = tokenize(i[3])
        tmp_list = list(nltk.trigrams(token_list))
        for j in tmp_list:
            trigrams_list.append(j)
        for k in token_list:
            skills_list.append(k)
        
    
    skill_dict={}
    trigram_freq = {}
    for l in trigrams_list:
        if l in trigram_freq:
            trigram_freq[l]+=1
        else:
            trigram_freq[l]=1
    for k in skills_list:
        if k in skill_dict:
            skill_dict[k]+=1
        else:
            skill_dict[k]=1
            
        
    # uncomment below print line to see the frequency for each trigram
    # Sorting the dictionary below to easily see and get the top values when needed. For trigrams
    sorted_freq = sorted(skill_dict.items(), key=operator.itemgetter(1),reverse=True)
    tri_education = sorted(trigram_freq.items(), key=operator.itemgetter(1),reverse=True)
    print("\n\nThese are preferred Education details for the position.\n")
    for j in range(0,8):
        print(tri_education[j])
    
    custom = ["years","year","na","less","than","and","in","of","data","technology"]
    for i in range(0,8):
        if(sorted_freq[i][0] not in custom):
            print(sorted_freq[i])
    
    
    print("\n##############################################\n")
    print("\nWork authorization requirement for this position in USA\n")
    #--------------------------------------------
    # now for 10th column i.e. skills.
    #--------------------------------------------
    auth_list=[]
    na = ["NA","na"]
    for i in rows:
        auth_list.append(i[4])
    auth_dict = {}
    for i in auth_list:
        if i in auth_dict:
            auth_dict[i]+=1
        else:
            auth_dict[i]=1
    sort_auth =  sorted(auth_dict.items(), key=operator.itemgetter(1),reverse=True)
    for j in range(0,3):
        if (sort_auth[j][0] not in na):
            print(sort_auth[j][0])
            
    return return_list

In [46]:
def description_analysis(input_file):
    trigrams_list =[]
    bigrams_list=[]
    # write your code here
    with open(input_file, "r",encoding="ISO-8859-1") as f:
        reader=csv.reader(f, delimiter=',')
        rows=[(row[8]) for row in reader]
    text=". ".join(rows).lower()
    
    #print(row_len)
    for i in rows:
        token_list = tokenize(i)
        tmp_list = list(nltk.trigrams(token_list))
        tmp_big_list = list(nltk.bigrams(token_list))
        #print(tmp_big_list)
        for j in tmp_list:
            trigrams_list.append(j)
        for l in tmp_big_list:
            bigrams_list.append(l)
    #print(bigrams_list)      
    finder = BigramCollocationFinder.from_words(\
        nltk.word_tokenize(text))
    print(token_list[0:10])
    finder.apply_freq_filter(5)
    print(finder.nbest(bigram_measures.pmi, 10))


    
if __name__ == "__main__":
    #this is for job title analysis
    file1 = "data_science.csv"
    file2 = "project_lead.csv"
    file3 = "vice_president.csv"
    
    

In [47]:
training_data_science = performance_evaluate(file1)



This analysis is for  ('data', 'scientist', 'intern')

##############################################

'input resume' can have more chances to become   ('data', 'scientist', 'intern')  if it has following job experience terms

('data', 'scientist', 'data')
('scientist', 'data', 'scientist')
('data', 'scientist', 'intern')
('scientist', 'data', 'analyst')
('data', 'analyst', 'data')
('analyst', 'data', 'analyst')
('senior', 'data', 'scientist')
('data', 'analyst', 'intern')
('scientist', 'data', 'modeler')
('graduate', 'research', 'assistant')


OR----####----####----####


('data', 'scientist')
('scientist', 'data')
('data', 'analyst')
('scientist', 'intern')
('research', 'assistant')
('analyst', 'data')
('data', 'modeler')
('software', 'engineer')
('intern', 'data')
('engineer', 'data')

##############################################



These are preferred skills for the position.

python 398
sql 383
data 365
learning 245
analysis 201
machine 193
hadoop 178
excel 125
apache 117
java

In [48]:
    training_software_engineer = performance_evaluate(file2)



This analysis is for  ('senior', 'software', 'engineer')

##############################################

'input resume' can have more chances to become   ('senior', 'software', 'engineer')  if it has following job experience terms

('senior', 'software', 'engineer')
('software', 'engineer', 'senior')
('engineer', 'senior', 'software')
('sr', 'software', 'engineer')
('software', 'engineer', 'software')
('engineer', 'software', 'engineer')
('software', 'engineer', 'project')
('software', 'engineer', 'sr')
('engineer', 'project', 'lead')
('engineer', 'sr', 'software')


OR----####----####----####


('software', 'engineer')
('senior', 'software')
('engineer', 'senior')
('sr', 'software')
('project', 'lead')
('engineer', 'software')
('engineer', 'project')
('project', 'manager')
('software', 'developer')
('engineer', 'sr')

##############################################



These are preferred skills for the position.

sql 138
java 78
server 65
oracle 61
net 53
web 51
javascript 42
ms 39


In [49]:
    training_vice_president = performance_evaluate(file3)



This analysis is for  ('vice', 'president', 'valley')

##############################################

'input resume' can have more chances to become   ('vice', 'president', 'valley')  if it has following job experience terms

('vice', 'president', 'vice')
('president', 'vice', 'president')
('assistant', 'vice', 'president')
('vice', 'president', 'senior')
('vice', 'president', 'director')
('vice', 'president', 'sales')
('vice', 'president', 'assistant')
('vice', 'president', 'na')
('president', 'assistant', 'vice')
('vice', 'president', 'associate')


OR----####----####----####


('vice', 'president')
('president', 'vice')
('assistant', 'vice')
('president', 'senior')
('president', 'director')
('president', 'sales')
('president', 'assistant')
('president', 'na')
('president', 'associate')
('project', 'manager')

##############################################



These are preferred skills for the position.

management 90
microsoft 68
office 62
excel 54
development 34
sales 33
word 29

In [50]:
    #this is for calculating avg job experience for data science
    print("\n##############################################\n")
    #dateToSum("indeed_scraped_data_science.csv")

    tot = len(training_data_science)
    #print(input_tokens)
    #print(len(training_data_science))
    print("\n##############################################\n")
    
    #these functions will analyze the job description of all positions from training dataset.
    
    print("\nBelow are the preferred job experience terms for the data science position.\n")
    description_analysis(file1)
    print("\n###################")
    
    print("\nBelow are the preferred job experience terms for the project lead position.\n")
    description_analysis(file2)
    print("####################")
    
    print("\nBelow are the preferred job experience terms for the vice president position.\n")
    description_analysis(file3)
    


##############################################

14021
Probability of the input resume of being data science is: 

##############################################



Below are the preferred job experience terms for the given position.

['utilize', 'lcms', 'analyze', 'target', 'analytes', 'assist', 'diagnosis', 'biochemical', 'disorders', 'utilize']
[('diabetic', 'retinopathy'), ('dow', 'jones'), ('higgs', 'boson'), ('abn', 'amro'), ('learn/', 'scipy/'), ('rac/10g', 'rac/9i'), ('scipy/', 'numpy/'), ('seed/production', 'fermenters'), ('tremendous', 'pride'), ('lin', 'z.')]
##################3
['lmm', 'legacy', 'managed', 'market', 'cm', 'customer', 'master', 'remediation', 'project', 'multiple']
[('ezx', 'xwindows'), ('jd', 'edwards'), ('proving', 'grounds'), ('rhode', 'island'), ('basking', 'ridge'), ('condor', 'arinc-429'), ('td', 'ameritrade'), ('daimler', 'chrysler'), ('loosely', 'coupled'), ('checkfree', 'apl')]
####################
['cmu', 'engineering', 'club', 'formula', 'sae', 't

In [51]:
def compare_prob(input_tokens,training_data_science):
    count = 0
    for itk in training_data_science:
        j = 0
        itk = itk[0].lower()
        #print(itk)
        for tdk in input_tokens:
            j = j + 1
            tdk = tdk.lower()
            #print(tdk)
            if(itk == tdk):
                count = count + (tot-j)    

    return count

tmp1 = compare_prob(input_tokens,training_data_science)
tmp2 = compare_prob(input_tokens,training_software_engineer)
tmp3 = compare_prob(input_tokens,training_vice_president)

tot = tmp1+tmp2+tmp3

#input sample is taken from with the skills of Project Lead. 

print("Chances of input resume to be a Data Scientist (CNN on job description) are: ")
print(tmp1/tot * 100)

print("\n\nChances of input resume to be a Project Lead (CNN on job description) are: ")
print(tmp2/tot * 100)

print("\n\nChances of input resume to be a Vice President (CNN on job description) are: ")
print(tmp3/tot * 100)


Chances of input resume to be a Data Scientist (CNN on job description) are: 
32.10631807763958
Chances of input resume to be a Project Lead (CNN on job description) are: 
38.37256176994119
Chances of input resume to be a Vice President (CNN on job description) are: 
29.521120152419222
